## Read input data and turn it into data frame

In [8]:
# Dependancies
import csv
import os
import pandas as pd 

In [9]:
# Supporting functions
def get_filename(ticker):
    return os.path.join("..", "InputData", ticker + ".csv")

In [10]:
# Read input data into dataframe
def read_ticker(file_name):
    # Input file into dataframe
    rawdata = pd.read_csv(file_name) 
    # Date into date type, date of week, week of year, Gain
    rawdata.Date = rawdata.Date.astype('datetime64[ns]')
    # isocalendar reports end of the year as 1st week - see 12/31/2008
    #rawdata['YearWeek'] = [ td.year + td.isocalendar()[1] / 100 for td in rawdata.Date] 
    rawdata['YearWeek'] = [ td.year + (int(td.strftime("%W")) + 1) / 100 for td in rawdata.Date]
    rawdata['Weekday'] = [ td.weekday() for td in rawdata.Date]
    rawdata['GainPct'] = (rawdata.Close - rawdata.Open) / rawdata.Open
    
    # Pivot data into new df
    return rawdata.pivot(index='YearWeek', columns='Weekday', values='GainPct')

In [11]:
# Sum data
def sum_ticker(ticker):
    df = read_ticker(get_filename(ticker))
    result = pd.DataFrame([df.sum(axis = 0, skipna = True) * 100])
    result.insert(0, "Ticker", ticker)
    return result

In [12]:
# Sum tickers
tickers = ["XLB","XLC","XLE","XLF","XLI","XLK","XLP","XLRE", "XLU", "XLV", "XLY"]

df = pd.DataFrame()

for ticker in tickers:
    print(ticker)
    df = df.append(sum_ticker(ticker))

df

XLB
XLC
XLE
XLF
XLI
XLK
XLP
XLRE
XLU
XLV
XLY


Weekday,Ticker,0,1,2,3,4
0,XLB,-82.356456,28.381947,26.159974,-18.542087,18.121169
0,XLC,-15.473824,2.916800,-7.533318,6.417480,-4.809059
0,XLE,-55.187244,32.999256,20.661226,-52.506902,28.039777
0,XLF,-46.049631,21.714262,28.158104,-22.468172,-10.608836
0,XLI,-55.539073,-10.658803,-13.242306,47.345461,-18.660919
0,XLK,-7.890154,-29.223387,19.537709,15.260011,-81.104524
0,XLP,34.290011,25.465714,11.637114,17.409478,-29.669799
0,XLRE,-3.517596,-5.247508,-11.046816,3.288574,8.655779
0,XLU,16.480246,-82.959038,-69.269637,32.023397,-4.258002
0,XLV,-67.831643,3.684168,31.500851,19.477674,-6.025189


In [13]:
# Total sum
total = pd.DataFrame([df.sum(axis = 0, skipna = True)])
total = total.drop(['Ticker'], axis=1)
total

Weekday,0,1,2,3,4
0,-322.119261,32.028042,58.388221,94.444988,-114.263089


In [20]:
# Add column with maximum gain of the week
middata = read_ticker(get_filename("XLE"))
middata['MaxGainDay'] = middata.idxmax(axis=1)
middata.head()

Weekday,0,1,2,3,4,MaxGainDay
YearWeek,,,,,,
2000.02,-0.027460,-0.009501,0.023923,0.038551,0.014108,3
2000.03,-0.008347,0.001122,-0.003367,0.010627,-0.001112,3
2000.04,NaN,0.019058,0.000000,-0.003300,0.005400,1
2000.05,-0.041622,-0.004444,-0.008989,-0.023729,-0.009281,1
2000.06,0.042959,-0.002288,0.004005,-0.010274,-0.019790,0


In [21]:
# Count days with highest weekly gains / minimal loses
middata.groupby('MaxGainDay').size()

MaxGainDay
0    197
1    201
2    237
3    207
4    187
dtype: int64

In [ ]:
# Sum gains total in %
